World Coins: A collection of coin images from 32 different currencies.

In [ ]:
# Seznam knihoven
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import os
import numpy as np
import cv2
import kaggle
import random
import concurrent.futures
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import os
import random
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Cesty k datasetu
data_dir_test = "data/coins/data/test"

# Načtení mapy kategorií
with open("/mnt/data/cat_to_name.json", "r") as f:
    cat_to_name = json.load(f)

# Seznam všech tříd (složek)
categories = os.listdir(data_dir_test)

# Náhodný výběr 100 obrázků
selected_images = []
for _ in range(100):
    category = random.choice(categories)
    category_path = os.path.join(data_dir_test, category)
    if os.path.isdir(category_path):
        image_file = random.choice(os.listdir(category_path))
        selected_images.append((os.path.join(category_path, image_file), category, image_file))

# Načtení vlastního předtrénovaného modelu
model = load_model("fine_tuned_resnet50_model.keras")

# Funkce pro klasifikaci obrázků
def classify_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalizace
    
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    
    return predicted_class

# Klasifikace vybraných obrázků
results = []
for img_path, category, file_name in selected_images:
    predicted_class = classify_image(img_path)
    category_info = cat_to_name.get(str(predicted_class), "Unknown,Unknown,Unknown").split(",")
    results.append([file_name, category, category_info[0], category_info[1], category_info[2]])

# Vytvoření datového rámce
columns = ["Filename", "True Category", "Predicted Coin", "Currency", "Country"]
df_results = pd.DataFrame(results, columns=columns)

# Zobrazení výsledků
import ace_tools as tools
tools.display_dataframe_to_user(name="Classification Results", dataframe=df_results)